<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/ImageNet_Dataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ImageNet (ILSVRC2012)

It contains 1000 classes, 1.28 million training images, and 50 thousand validation images. There are 1,281,167 images and 732-1300 per class in the ILSVRC2012 training set. This dataset spans 1000 object classes and contains 1,281,167 training images, 50,000 validation images and 100,000 test images. It requires more than 150GB of storage, and training a resnet50 on it will take around 215 hours using a T4 GPU on Google Colab. Folder name to actual class mapping: https://www.image-net.org/challenges/LSVRC/2012/browse-synsets.php <br>
Sample size is not equal in ImageNet. For example top 10 classes:<br>
n02094433:    3047 (Yorkshire terrier)<br>
n02086240:    2563 (Shih-Tzu)<br>
n01882714:    2469 (koala bear, kangaroo bear, native bear, )<br>
n02087394:    2449 (Rhodesian ridgeback)<br>
n02100735:    2426 (English setter)<br>
n00483313:    2410 (singles)<br>
n02279972:    2386 (monarch butterfly, Danaus plexippus)<br>
n09428293:    2382 (seashore)<br>
n02138441:    2341 (meerkat)<br>
n02100583:    2334 (vizsla, Hungarian pointer)<br>


Task-1. Image classification (2010-2014): Algorithms produce a list of object categories present in the image.<br>
Task-2. Single-object localization (2011-2014): Algorithms
produce a list of object categories present in the image, along with an axis-aligned bounding box indicating the position and scale of one instance of each object category.<br>
Task-3. Object detection (2013-2014): Algorithms produce
a list of object categories present in the image along
with an axis-aligned bounding box indicating the
position and scale of every instance of each object
category.<br>

#Download Links:

Training Images (taskl&2): https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_train.tar <br>
Training Annotations (taskl&2): https://image-net.org/data/ILSVRC/2012/ILSVRC2012_bbox_train_v2.tar.gz <br>

Validation Images (all tasks): https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar

Validation Annotations (all tasks): https://image-net.org/data/ILSVRC/2012/ILSVRC2012_bbox_val_v3.tgz


# Preparing Train Images into Folders (Not using in this tutorial)
src: https://github.com/pytorch/examples/blob/main/imagenet/extract_ILSVRC.sh

In [ ]:
# Create train directory; move .tar file; change directory
!mkdir imagenet/train && mv ILSVRC2012_img_train.tar imagenet/train/ && cd imagenet/train
# Extract training set; remove compressed file
!tar -xvf ILSVRC2012_img_train.tar && rm -f ILSVRC2012_img_train.tar
#
# At this stage imagenet/train will contain 1000 compressed .tar files, one for each category
#
# For each .tar file: 
#   1. create directory with same name as .tar file
#   2. extract and copy contents of .tar file into directory
#   3. remove .tar file
!find . -name "*.tar" | while read NAME ; do mkdir -p "${NAME%.tar}"; tar -xvf "${NAME}" -C "${NAME%.tar}"; rm -f "${NAME}"; done

# Download only Validation Set

In [9]:
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar

--2022-10-05 18:40:55--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘ILSVRC2012_img_val.tar’

ILSVRC2012_img_val. 100%[===================>]   6.28G  38.2MB/s    in 2m 58s  

2022-10-05 18:43:53 (36.2 MB/s) - ‘ILSVRC2012_img_val.tar’ saved [6744924160/6744924160]



# Preparing Valid Images into Folders

In [13]:
!mkdir imagenet
!mkdir imagenet/val
!tar -xvf ILSVRC2012_img_val.tar --directory imagenet/val
%cd imagenet/val
!wget -qO- https://raw.githubusercontent.com/soumith/imagenetloader.torch/master/valprep.sh | bash
%cd ../..

In [25]:
import argparse
import os
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

valdir = os.path.join('imagenet', 'val')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])


val_dataset = datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]))
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=32, shuffle=False,
    num_workers=2, pin_memory=True)

for i, (input, target) in enumerate(val_loader):
    print(input.shape, target)
    break


torch.Size([32, 3, 224, 224]) tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
